In [288]:
import configparser
from pathlib import Path
import collections
import pandas as pd
import pandas.io.sql as pandasql
from psycopg2.extras import execute_values
from psycopg2 import connect
import numpy
import matplotlib as mpl
from IPython.display import HTML, display
import rick
import numpy as np
import matplotlib.pyplot as plt
from setuptools import setup, find_packages
CONFIG = configparser.ConfigParser()
CONFIG.read(str(Path.home().joinpath('db.cfg')))
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

In [381]:
merge_set = pd.read_sql('''select rank, start_vid, end_vid, length, link_set, new_start_vid, new_end_vid, new_length, new_link_set, st_hausdorffdistance  from test_route_ranks order by length
                          ''',con)

In [397]:
def check_usage(link_set, link_used):
    result = []
    #print('link_set' +str(link_set))
    #print('link_used' +str(link_used))
    if link_used == []:
        return False
    for i in link_used:
        if collections.Counter(link_set) == collections.Counter(i):
            result.append(True)
        else:
            result.append(False)
    if True in result:
        return True
    else:
        return False

In [387]:
link_set = [7545686260, 7545686270, 7545655610, 295886600]

In [394]:
True in check_usage(link_set, link_used)

True

In [404]:
link_used = []
new_sets = []
rows = []
unique_set = merge_set[['start_vid', 'end_vid']].drop_duplicates()

# loop through unique sets of intersection
# !!this will be ordered by priority and length eventually!!
for index, key in unique_set.reset_index(drop=True).iterrows(): 
    
    start_vid = unique_set['start_vid'].iloc[index]
    end_vid = unique_set['end_vid'].iloc[index]
    print(start_vid, end_vid)
    prepare_set = merge_set[(merge_set['start_vid'] == start_vid) & (merge_set['end_vid'] == end_vid)]
    link_set = prepare_set['link_set'].iloc[0]
    
    # Check if this link_set was used to match another set before
    # if not, selection best and merge
    if check_usage(link_set, link_used) == False:
        print('not used')
       
        # if there are more than one rank and length is less than 100 or something         
        if prepare_set['length'].iloc[0] < 100:    
            # if there are two possibility for this intersection set
            # then compare which one is better

            if prepare_set['rank'].shape[0] == 2: 
                rank1=prepare_set[(merge_set['rank']==1)]
                rank2=prepare_set[(merge_set['rank']==2)]

                # find hausdorffdistance difference and length difference
                hausorffdiff = abs(rank1['st_hausdorffdistance'].iloc[0]/rank2['st_hausdorffdistance'].iloc[0])
                lengthdiff = rank1['new_length'].iloc[0] - rank2['new_length'].iloc[0]

                # make selection based on hausdorffdistance difference and length difference
                # could try and play with the params a little bit
                if hausorffdiff < 5 and lengthdiff > 20:
                    select_set = 2
                else:
                    select_set = 1

                selection = merge_set[(merge_set['rank']==select_set)&(merge_set['start_vid']==start_vid)&(merge_set['end_vid']==end_vid)]
                
                # find links that were used to merge, append it to another list
                # also have to find out if it was using a set of link that was used before
                old_links = selection['link_set'].iloc[0]
                new_links = selection['new_link_set'].iloc[0]
                additional_links = list(set(new_links).difference(old_links))
                
                if check_usage(additional_links,link_used) == False:
                    new_set = np.array([selection['new_start_vid'].iloc[0], selection['new_end_vid'].iloc[0], selection['new_length'].iloc[0], selection['new_link_set'].iloc[0]])
                    link_used.append(additional_links)
                    link_used.append(old_links)
                    new_sets.append(new_set)
                else:
                    new_set = np.array([prepare_set['start_vid'].iloc[0], prepare_set['end_vid'].iloc[0], prepare_set['length'].iloc[0], prepare_set['link_set'].iloc[0]])
                    link_used.append(old_links)
                    new_sets.append(new_set)
                    
            # if there is only one possibility
            # see if that possibility is viable 
            elif prepare_set['rank'].shape[0] == 1:
                selection = merge_set[(merge_set['rank']==1)&(merge_set['start_vid']==start_vid)&(merge_set['end_vid']==end_vid)]
                old_links = selection['link_set'].iloc[0]
                new_links = selection['new_link_set'].iloc[0]
                additional_links = list(set(new_links).difference(old_links))
                
                if check_usage(additional_links, link_used) == False:
                    new_set = np.array([selection['new_start_vid'].iloc[0], selection['new_end_vid'].iloc[0], selection['new_length'].iloc[0], selection['new_link_set'].iloc[0]])
                    link_used.append(old_links)
                    link_used.append(additional_links)
                    new_sets.append(new_set)
                else:
                    new_set = np.array([prepare_set['start_vid'].iloc[0], prepare_set['end_vid'].iloc[0], prepare_set['length'].iloc[0], prepare_set['link_set'].iloc[0]])
                    print('was used before')
                    link_used.append(old_links)
                    new_sets.append(new_set)
                    
        # if the set length is longer than 100 then 
        # check to see if it was used to merge 
        # if yes dont append, if not append
        ######### links that are longer than 100 and used were not getting appended to the used list 
        else: 
            print('longer than 100m')
            old_set = np.array([prepare_set['start_vid'].iloc[0], prepare_set['end_vid'].iloc[0], prepare_set['length'].iloc[0], prepare_set['link_set'].iloc[0]])
            new_sets.append(old_set)
            link_used.append(link_set)
                
    # if link_set as used, continue and dont append
    else:
        print('used')

for i in range(len(new_sets)):
    item = new_sets[i]
    length = item[2].astype(float)

    row = (item[0].astype(float), item[1].astype(float), length, item[3])

    rows.append(row)
        
sql = '''insert into congestion.test_shortseg(start_vid, end_vid, length, link_set) VALUES %s'''    
with con:
    with con.cursor() as cur:
        execute_values(cur, sql, rows)    

        

30415730 30415731
not used
30415731 30415730
not used
30414528 30414578
not used
30414575

/etc/jupyterhub/.venv/lib/python3.5/site-packages/ipykernel_launcher.py:27: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/etc/jupyterhub/.venv/lib/python3.5/site-packages/ipykernel_launcher.py:28: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



 30414573
not used
30414576 30414577
not used
30414577 30414576
not used
30414685 30414684
not used
30414684 30414685
not used
30363537 30415081
not used
30415081 30363537
not used
30414650 30414649
not used
30414649 30414650
not used
30414534 30414531
not used
30414531 30414534
not used
30414968 30414969
not used
30414969 30414968
not used
30414973 30414993
not used
30414993 30414973
not used
30414551 30414540
not used
30414540 30414551
not used
30362987 862292856
not used
862292856 30362987
not used
30414959 30414958
not used
30414958 30414959
not used
30414919 30414918
not used
30414866 30414921
not used
30414921 30414866
not used
30415158 30415164
not used
30415032 30414932
not used
30415168 30415158
not used
30415175 30415170
not used
862292856 30363018
used
30363018 862292856
used
30415032 30415036
not used
30415036 30415032
not used
30415036 30414932
not used
30414932 30415036
not used
30415168 30415164
not used
30414894 30414892
not used
30415170 30415168
used
30414902 30414892

In [405]:
len(new_sets)

209